# Refining vectors in testing space

## Aims
1. Is there a better lens to look at the data with? It doesn't necessarily have to relate to the metric.
    1. We could use dimensionality reduction on the fingerprints e.g. PCA/MDS and then use 2D values of that to be the lens.
    1. There are problems with discretised lenses because they naturally fall into bins anyway and this complicates designing a cover (that's why yesterday I had to use lots of overlap)
1. What does the distance matrix mean? 
1. What is a meaningful thing to colour by? We could use "is active/inactive against target B"

## Pitfalls
1. Lots of drugs have been tested against only one target and therefore show up as identical in the metric. Is there a better way to do that?


In [1]:
import rdkit
import rdkit.Chem as Chem
import rdkit.Chem.AllChem as AllChem
import kmapper as km
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from collections import Counter
import sys

In [2]:
ACTIVITY_CUTOFF = 5.0
DESIRED_TARGET = "CHEMBL218"
DESIRED_TARGET2 = "CHEMBL4015"
DESIRED_TARGET3 = "CHEMBL220"

In [3]:
with open("../data/processed/curated_set_with_publication_year.pd.pkl", "rb") as infile:
    df = pickle.load(infile)

possible_targets = Counter([item for item in df["TGT_CHEMBL_ID"]])
print(possible_targets.most_common(50))
possible_drugs = Counter([item for item in df["CMP_CHEMBL_ID"]])
vector_df = pd.DataFrame(0, columns=possible_drugs.keys(), index=possible_targets.keys(), dtype=np.int8)

[('CHEMBL240', 4703), ('CHEMBL253', 3472), ('CHEMBL218', 2997), ('CHEMBL251', 2976), ('CHEMBL228', 2853), ('CHEMBL264', 2548), ('CHEMBL226', 2544), ('CHEMBL217', 2473), ('CHEMBL344', 2358), ('CHEMBL243', 2315), ('CHEMBL256', 2304), ('CHEMBL205', 2257), ('CHEMBL279', 2142), ('CHEMBL261', 2089), ('CHEMBL4235', 2020), ('CHEMBL244', 2010), ('CHEMBL222', 2003), ('CHEMBL233', 1998), ('CHEMBL4078', 1994), ('CHEMBL284', 1950), ('CHEMBL237', 1908), ('CHEMBL259', 1828), ('CHEMBL4822', 1799), ('CHEMBL3371', 1773), ('CHEMBL214', 1703), ('CHEMBL313', 1690), ('CHEMBL3594', 1678), ('CHEMBL203', 1659), ('CHEMBL224', 1643), ('CHEMBL4296', 1594), ('CHEMBL260', 1589), ('CHEMBL235', 1575), ('CHEMBL234', 1569), ('CHEMBL225', 1565), ('CHEMBL236', 1550), ('CHEMBL220', 1542), ('CHEMBL238', 1518), ('CHEMBL247', 1474), ('CHEMBL255', 1445), ('CHEMBL3952', 1424), ('CHEMBL2039', 1403), ('CHEMBL340', 1386), ('CHEMBL3242', 1380), ('CHEMBL204', 1347), ('CHEMBL5071', 1332), ('CHEMBL239', 1324), ('CHEMBL325', 1298), ('

In [18]:
counted = 0
for index, row in df.iterrows():
    drug = row["CMP_CHEMBL_ID"]
    target = row["TGT_CHEMBL_ID"]
    if not counted % 10000:
        print("Counted up to", counted)
    if row["BIOACT_PCHEMBL_VALUE"] > ACTIVITY_CUTOFF:
        vector_df[drug][target] = 1
    else:
        vector_df[drug][target] = -1
    counted += 1

Counted up to 0
Counted up to 10000
Counted up to 20000
Counted up to 30000
Counted up to 40000
Counted up to 50000
Counted up to 60000
Counted up to 70000
Counted up to 80000
Counted up to 90000
Counted up to 100000
Counted up to 110000
Counted up to 120000
Counted up to 130000
Counted up to 140000
Counted up to 150000
Counted up to 160000
Counted up to 170000
Counted up to 180000
Counted up to 190000
Counted up to 200000
Counted up to 210000
Counted up to 220000
Counted up to 230000
Counted up to 240000
Counted up to 250000
Counted up to 260000
Counted up to 270000
Counted up to 280000
Counted up to 290000
Counted up to 300000
Counted up to 310000


In [17]:
fingerprint_df = pd.Series(index=np.array([row["CMP_CHEMBL_ID"] for _, row in df.iterrows()], dtype=rdkit.DataStructs.cDataStructs.ExplicitBitVect))


In [11]:
counted = 0
fingerprint_dict = {}
for index, row in df.iterrows():
    drug = row["CMP_CHEMBL_ID"]
    target = row["TGT_CHEMBL_ID"]
    if target in [DESIRED_TARGET, DESIRED_TARGET2]:
        fingerprint = AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(row["SMILES"]),3)
        fingerprint_dict[drug] = AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(row["SMILES"]),3)
    else:
        pass
    counted += 1
    if not counted % 1000:
        print(counted)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [20]:
vector_df

,CHEMBL19831,CHEMBL332157,CHEMBL52427,CHEMBL353213,CHEMBL100672,CHEMBL99622,CHEMBL256892,CHEMBL101041,CHEMBL168411,CHEMBL47207,...,CHEMBL3084704,CHEMBL3084711,CHEMBL3084715,CHEMBL3084707,CHEMBL3084710,CHEMBL3084717,CHEMBL3084713,CHEMBL3304900,CHEMBL3274623,CHEMBL3305786
CHEMBL209,1,1,1,1,1,1,1,1,-1,1,...,0,0,0,0,0,0,0,0,0,0
CHEMBL203,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEMBL3646,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEMBL4315,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEMBL5720,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEMBL1878,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEMBL243,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEMBL4552,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEMBL244,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEMBL1929,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
def dissimilarity(vec1, vec2, metric="euclidean"):
    """
    Takes in two vectors with values +1 and -1. Computes how far away they are
    in according to the metric.
    Current metrics:
    - Euclidean, computes sqrt(vec1 dot vec2)
    - Cosine distance
    - Tanimoto
    """
    metric = metric.lower()
    if metric == "euclidean":
        distance = np.abs(vec1 - vec2)
        return np.sqrt(np.dot(distance, distance))
    
    if metric == "cosine":
        return 1.0 - np.dot(vec1, vec2)/(np.sqrt(np.dot(vec1, vec1)) * np.sqrt(np.dot(vec2, vec2)))
                                   
    if metric == "tanimoto":
        return 1.0 - np.dot(vec1, vec2) / (np.dot(vec1, vec1) + np.dot(vec2, vec2) - np.dot(vec1, vec2))

In [6]:
with open("../data/processed/vector_df.pd.pkl", "wb") as outfile:
    pickle.dump(vector_df, outfile)

In [19]:
mask = vector_df.loc[DESIRED_TARGET].values != 0
mask2 = vector_df.loc[DESIRED_TARGET2].values != 0
mask3 = vector_df.loc[DESIRED_TARGET3].values != 0
sub_df = (vector_df.T[mask | mask2])
sub_df.shape
sub_df

,CHEMBL209,CHEMBL203,CHEMBL3646,CHEMBL4315,CHEMBL5720,CHEMBL1878,CHEMBL243,CHEMBL4552,CHEMBL244,CHEMBL1929,...,CHEMBL2021745,CHEMBL4079,CHEMBL1293292,CHEMBL1293287,CHEMBL4000,CHEMBL5080,CHEMBL3085613,CHEMBL2439944,CHEMBL1075152,CHEMBL3246
CHEMBL116463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEMBL1683055,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEMBL226066,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEMBL1683063,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEMBL1683060,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEMBL1683069,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEMBL1683071,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEMBL1683072,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEMBL209899,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEMBL1739922,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
count = 0
distance_matrix = np.zeros([len(sub_df), len(sub_df)])
with open("distance_matrix.csv", "w") as fi:
    for drug_index in range(len(sub_df)):
        if not drug_index % 100:
            print(drug_index)
        drug = sub_df.iloc[drug_index].values
        for other_index in range(drug_index):
            other_drug = sub_df.iloc[other_index].values
            distance = dissimilarity(drug, other_drug, "tanimoto")
            distance_matrix[drug_index, other_index] = distance
            distance_matrix[other_index, drug_index] = distance

0
16
2
2
2
2
2
2
2
3
1
2
2
2
2
2
2
2
2
3
3
2
3
2
3
4
4
2
2
2
2
2
2
4
2
2
2
1
7
-1
2
3
2
0
1
3
3
3
3
3
3
3
1
4
3
3
4
4
4
1
1
1
1
1
1
1
1
1
1
2
1
2
2
1
1
2
-1
2
4
2
1
6
2
1
2
3
1
1
1
1
1
1
1
1
3
3
2
2
2
2
2
100
2
2
2
2
1
2
1
2
2
2
2
2
2
1
2
2
1
1
1
2
1
2
2
2
2
0
2
1
1
2
4
2
0
2
1
1
2
2
3
2
2
3
2
1
3
1
2
3
2
2
0
2
1
1
2
2
1
2
2
1
1
2
2
1
2
1
3
2
2
2
2
2
2
2
2
2
2
2
3
3
2
2
2
2
2
2
2
2
2
1
2
1
2
1
2
2
2
2
2
0
200
2
2
2
2
2
1
2
1
2
2
2
2
1
2
2
2
2
2
2
4
2
1
1
2
2
2
2
2
2
2
2
1
1
2
2
3
2
2
2
2
2
1
1
1
1
1
2
2
2
2
2
2
2
2
1
1
2
2
1
1
1
1
1
1
1
2
1
2
2
1
1
1
1
1
2
2
3
3
1
1
2
2
1
2
1
2
1
2
0
2
2
0
2
2
1
2
2
2
2
2
300
2
2
2
2
1
1
2
2
1
2
1
2
4
3
2
2
2
2
2
2
2
2
2
2
2
1
2
-1
1
1
1
2
2
2
3
2
2
2
1
2
2
4
1
1
1
1
2
1
1
1
2
1
1
4
1
2
1
1
2
4
2
1
1
1
1
2
2
1
1
2
2
0
0
1
2
1
2
1
1
1
1
1
2
2
1
3
1
1
1
2
1
1
1
1
2
2
2
1
2
2
400
2
2
1
1
1
1
2
1
2
2
1
2
2
-2
1
2
2
2
2
1
1
1
1
1
1
2
2
2
2
2
1
1
1
1
2
2
2
2
1
1
1
1
1
2
2
1
2
2
2
2
2
2
2
2
0
2
2
2
2
1
4
5
1
1
1
1
1
2
3
1
2
1
1
1
2
2
2
4
4
2
2
2
1
2
2
2
2
2
2

KeyboardInterrupt: 

In [ ]:
chemical_distance = np.zeros_like(distance_matrix)
for index in range(len(sub_df)):
    drug = sub_df.iloc[index]
    fingerprint = fingerprint_dict[drug.name]
    if not index % 100:
        print(index)
    for other_index in range(index):
        other_fingerprint = fingerprint_dict[sub_df.iloc[other_index].name]
        chem_dissimiliarity = 1.0 - rdkit.DataStructs.TanimotoSimilarity(fingerprint, other_fingerprint)
        chemical_distance[index, other_index] = chem_dissimiliarity
        chemical_distance[other_index, index] = chem_dissimiliarity

In [24]:
from sklearn.manifold import MDS
print(sq_distance_matrix.shape)
lens = MDS(n_components=2, dissimilarity="precomputed", metric=False).fit_transform(sq_distance_matrix)

(5773, 5773)


In [25]:
color_func = np.array([sub_df.T[drug][DESIRED_TARGET] for drug in sub_df.T])


In [26]:
import hdbscan
from IPython.display import SVG, IFrame
mapper = km.KeplerMapper(verbose=1)
graph = mapper.map(lens,
                   X=distance_matrix,
                   precomputed=True,
                   cover=km.Cover(n_cubes=[8, 8], perc_overlap=0.38),
                   clusterer=hdbscan.HDBSCAN(metric='precomputed', min_cluster_size=5, min_samples=3))
mapper.visualize(graph, path_html="2019-04-05-mb-vectors-in-drug-testing-space-mds.html",
                 title="Vectors in Drug Testing Space With MDS", color_function=color_func)
IFrame("2019-04-05-mb-vectors-in-drug-testing-space-mds.html", 800, 600)

KeplerMapper(verbose=1)
Mapping on data shaped (5755, 5755) using lens shaped (5773, 2)



ValueError: all the input array dimensions except for the concatenation axis must match exactly